In [1]:
# %run development/libraries/placements-proxy

In [2]:
%run ./data-source-dbrotate

In [3]:
import sys
if sys.version_info[0] < 3:
  from HTMLParser import HTMLParser
else:
  from html.parser import HTMLParser

In [4]:
from pyspark.ml.clustering import LDA, LocalLDAModel

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import hashlib
from pyspark.ml import *
from pyspark.ml import Pipeline
#from html.parser import HTMLParser
from math import sqrt as msqrt
from math import isnan
import numpy
import re
from pyspark.ml.feature import *
from pyspark.ml.classification import *
from pyspark.ml.clustering import *
from pyspark.ml.evaluation import *
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, IDF, CountVectorizerModel, IDFModel
from pyspark.ml.clustering import LDA, LocalLDAModel
from pyspark import keyword_only
from pyspark.ml.param.shared import *
from datetime import datetime
import random
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
def remove_html(anystring):
  if anystring:
    parser=HTMLParser()
    anystring = parser.unescape(anystring)
    anystring=re.sub("<.*?>"," ",anystring)
    #anystring = re.sub("&\w+;"," ",anystring)
    anystring=re.sub("\xa0"," ",anystring)
  return anystring

remove_html = udf(remove_html , StringType())

In [7]:
country = 'hk'
train = spark.read.parquet("s3a://exploratory/"+country+"/data/train_v2")

train.count()

Out[ 46 ]: 274807

In [8]:
def process_profile_text(JsGlobal_JobSeekerWorkExperience_Position, JsGlobal_JobSeekerWorkExperience_JobDuties , JsGlobal_JobSeekerSkill_Skill):
  text = ''
  if JsGlobal_JobSeekerWorkExperience_Position and JsGlobal_JobSeekerWorkExperience_JobDuties: 
    if len(JsGlobal_JobSeekerWorkExperience_Position) == len(JsGlobal_JobSeekerWorkExperience_JobDuties):
      for i in range(len(JsGlobal_JobSeekerWorkExperience_Position)):
        text += JsGlobal_JobSeekerWorkExperience_Position[i] + JsGlobal_JobSeekerWorkExperience_JobDuties[i] + ' '
    else:
      text = ' '
  if JsGlobal_JobSeekerSkill_Skill:
    text += "," . join(JsGlobal_JobSeekerSkill_Skill)     
  return text

process_profile_text = udf(process_profile_text , StringType())

In [9]:
# from pyspark.ml.feature import HashingTF, IDF, Tokenizer
# from pyspark.ml.feature import StopWordsRemover
# from pyspark.ml import Pipeline


# regex_tokenizer = RegexTokenizer(inputCol='text', outputCol="tokenized_text" , pattern="[\s,(),\W+]", minTokenLength=2)
# removal = StopWordsRemover(inputCol="tokenized_text", outputCol="text_filtered")
# cv = CountVectorizer(inputCol='text_filtered', outputCol = 'desc_count', vocabSize = 50000, minDF = 3) 
    # ** word count of the words in each document (Transforms text into a sparse matrix of n-gram counts)
# idf = IDF(inputCol = "desc_count", outputCol = "desc_features")
    # ** Performs the TF-IDF (term frefuency and inverse documents frequency) transformation from a provided matrix of
# lda = LDA(k = 500, featuresCol = 'desc_features', maxIter = 600)

# stages = [regex_tokenizer, removal, cv, idf, lda]

# pipeline = Pipeline(stages=stages)
# ldaModel = pipeline.fit(text_df)

In [10]:
class PlacementsProxyLDAHandler:
  
  def load_pipeline(self):
    #load the LDA Model
    ldaModel = LocalLDAModel.load(self.lda_model_path)
    #load the CountVectorizer Model
    cvmodel=CountVectorizerModel.load(self.cv_model_path) 
    #Load the IDF model
    idfModel=IDFModel.load(self.idf_model_path)  counts
    #define the regex tokenizer
    regex_tokenizer = RegexTokenizer(inputCol="desc", outputCol="tokenized_text", pattern="[\s,(),\W+]",minTokenLength=2) #maybe want to change in future
    removal = StopWordsRemover(inputCol="tokenized_text", outputCol="text_filtered")
    #define the pipeline
    pipeline = Pipeline(stages=[regex_tokenizer, removal, cvmodel, idfModel, ldaModel])
    return pipeline
    
  def pipeline_fit_transform(self, df, lcol):
    #apply pipeline to df
    temp = df.withColumn("desc", col(lcol+'_desc'))
    temp = self.pipeline.fit(temp).transform(temp)
    temp = temp.withColumn(
      (lcol+'_topics'),
      col('topicDistribution')
    ).drop(
      'desc',
      'tokenized_text',
      'text_filtered',
      'desc_count',
      'desc_features',
      'topicDistribution'
    )
    return temp

  def __init__(self, cv_model_path=None, idf_model_path=None, lda_model_path=None):
    self.cv_model_path = cv_model_path
    self.idf_model_path = idf_model_path
    self.lda_model_path = lda_model_path
    self.pipeline = self.load_pipeline()
  
  
  def lda_dataset_preprocessing(self, df):
    concat_list = udf(lambda x: ", " . join(x) if x and len(x)>0 else '')
    #clean up data fields for use
    temp_df = df.fillna("").select(
      'Id',
      concat(
      'JobAd_JobTitle',
      remove_html('JobAd_JobDescriptionRequirement')
      ).alias('job_desc'),
      process_profile_text('JsGlobal_JobSeekerWorkExperience_Position' , 
       'JsGlobal_JobSeekerWorkExperience_JobDuties' , 
       'JsGlobal_JobSeekerSkill_Skill'
      ).alias('cand_desc')
    )
    return temp_df
  
  def transform(self, df):
    temp_df = self.lda_dataset_preprocessing(df)
    for item in ['job', 'cand']:
      temp_df = self.pipeline_fit_transform(temp_df, item)

    return temp_df


In [11]:
for i in range(len(stages)):
  print( i , ldaModel.stages[i])
  
  
# 0 RegexTokenizer_ca5c8ecf8bd3
# 1 StopWordsRemover_30460b775b2c
# 2 CountVectorizer_1154f3301dd4 ## ldaModel.stage[2]
# 3 IDF_4c9f9c07166c ## ldaModel.stage[3]
# 4 LDA_b3084c6c3b4b ## ldaModel.stage[4]

In [12]:
cv_model_path = "s3a://exploratory/hk/model/cv_v1"
idf_model_path = "s3a://exploratory/hk/model/idf_v1"
lda_model_path = "s3a://exploratory/hk/model/lda_v1"

In [13]:
ldaModel = PlacementsProxyLDAHandler(
  cv_model_path = cv_model_path, 
  idf_model_path = idf_model_path, 
  lda_model_path = lda_model_path
)

In [14]:
ldf_df = ldaModel.transform(train)

In [15]:
def cosine_similarity(vec1, vec2): #distance between 2 vectors
  dotprod = vec1.dot(vec2) # dot product of 2 vectors 
  norm1 = float(numpy.sqrt(vec1.dot(vec1)))
  norm2 = float(numpy.sqrt(vec2.dot(vec2)))
  if norm1 == 0 or norm2 == 0:
    return 0.0
  else:
    a = dotprod/(norm1*norm2)
    return a.item()
cosine_similarity = udf(cosine_similarity, FloatType())

def hellinger_distance(vec1, vec2): # distance between the probability vectors. (density-based distance)
  if (numpy.count_nonzero(vec1) == 0) or (numpy.count_nonzero(vec2) == 0):
    return 0.0
  else:
    _SQRT2 = numpy.sqrt(2)
    temp = 1 - numpy.sqrt(numpy.sum((numpy.sqrt(vec1) - numpy.sqrt(vec2)) ** 2)) / _SQRT2
    return temp.item() # to change it from a numpy float to an ordinary float
hellinger_distance = udf(hellinger_distance, FloatType())

In [16]:
ldf_df.columns

Out[ 50 ]: ['Id', 'job_desc', 'cand_desc', 'job_topics', 'cand_topics']

In [17]:
temps_ds = ldf_df.select(
    'Id',
    cosine_similarity(col('job_topics'),col('cand_topics')).alias('lda_cosine_similarity'),
    hellinger_distance(col('job_topics'),col('cand_topics')).alias('lda_hellinger_distance')
  )

In [18]:
display(temps_ds)

Id,lda_cosine_similarity,lda_hellinger_distance
100003062648106,0.38867724,0.1606794
100003062513585,0.71848863,0.3417426
100003063419850,0.4779005,0.21012911
100003062513654,0.53904,0.22841015
100003061249067,0.13816322,0.13377227
100003062712298,0.71625865,0.310738
100003062782349,0.3072503,0.22119874
100003061713435,0.13813016,0.077141024
100003061422341,0.1502766,0.12907057
100003060715057,3.5019693E-4,0.01749012


In [19]:
similarity_df = train.join(
  temps_ds,
  "Id",
  'left_outer'
)

display(similarity_df)

org.apache.spark.SparkException: Could not execute broadcast in 300 secs. You can increase the timeout for broadcasts via spark.sql.broadcastTimeout or disable broadcast join by setting spark.sql.autoBroadcastJoinThreshold to -1
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:197)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:164)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:152)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:183)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:152)
	at org.apache.spark.sql.execution.joins.BroadcastHashJoinExec.doExecute(BroadcastHashJoinExec.scala:73)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:143)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:183)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.ProjectExec.doExecute(basicPhysicalOperators.scala:80)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:143)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:183)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.collect.Collector$.collect(Collector.scala:66)
	at org.apache.spark.sql.execution.collect.Collector$.collect(Collector.scala:75)
	at org.apache.spark.sql.execution.ResultCacheManager.getOrComputeResult(ResultCacheManager.scala:497)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollectResult(limit.scala:48)
	at org.apache.spark.sql.Dataset.org$apache$spark$sql$Dataset$$collectResult(Dataset.scala:2822)
	at org.apache.spark.sql.Dataset$$anonfun$collectResult$1.apply(Dataset.scala:2813)
	at org.apache.spark.sql.Dataset$$anonfun$collectResult$1.apply(Dataset.scala:2812)
	at org.apache.spark.sql.Dataset$$anonfun$54.apply(Dataset.scala:3430)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:89)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:196)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:84)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:126)
	at org.apache.spark.sql.Dataset.withAction(Dataset.scala:3429)
	at org.apache.spark.sql.Dataset.collectResult(Dataset.scala:2812)
	at com.databricks.backend.daemon.driver.OutputAggregator$.withOutputAggregation0(OutputAggregator.scala:119)
	at com.databricks.backend.daemon.driver.OutputAggregator$.withOutputAggregation(OutputAggregator.scala:51)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$getResultBufferInternal$1.apply(PythonDriverLocal.scala:574)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$getResultBufferInternal$1.apply(PythonDriverLocal.scala:539)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:486)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.getResultBufferInternal(PythonDrive